In [1]:
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple

import torch
import torchvision
print(f"[INFO] torch version: {torch.__version__}")
print(f"[INFO] torchvision version: {torchvision.__version__}")

from torch import nn
from torch.utils.data import DataLoader

try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary

import os
NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32
SEED = 1161

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] running on {device}.")

[INFO] torch version: 2.5.1+cu121
[INFO] torchvision version: 0.20.1+cu121
[INFO] Couldn't find torchinfo... installing it.
[INFO] running on cuda.


In [2]:
import requests
import zipfile
import urllib.request

from pathlib import Path

# Setup path to data folder
image_path = Path("data/food-101-tiny")
file_name = "food101tiny.zip"

# If the image folder doesn't exist, download it and prepare it...
#if False:
if image_path.is_dir():
  print(f"Image directory exists: {image_path}. Continue with existing data.")
else:
  print(f"Image directory not found: {image_path}")
  print("Proceed to download food-101-tiny from Kaggle...")
  image_path.mkdir(parents=True, exist_ok=True)
  urllib.request.urlretrieve("https://www.kaggle.com/api/v1/datasets/download/msarmi9/food101tiny", file_name)
  with zipfile.ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall()
  print("Images created.")

Image directory not found: data/food-101-tiny
Proceed to download food-101-tiny from Kaggle...
Images created.


In [24]:
def create_effnetb2_model(num_classes: int=10,
                          seed: int=1161,
                          dropout: float=0.3):
  """Creates an EfficientNetB2 feature extractor model and transforms.

  Args:
      num_classes: number of classes in the classifier head.
          Defaults: 10 (size of food-101-tiny).
      seed: random seed value.
      dropout: dropout rate of the classifier head.

  Returns:
      model: EffNetB2 feature extractor model.
      transforms: EffNetB2 image transforms.
  """

  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights)

  # Freezes all layers in base model
  for param in model.parameters():
    param.requires_grad = False

  # Changes classifier head
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
      # 1408: number of in_feature of effnetb2.classifier
  )

  device = "cuda" if torch.cuda.is_available() else "cpu"
  return model.to(device), transforms

In [25]:
effnetb2, effnetb2_transforms = create_effnetb2_model(seed=SEED)

In [5]:
train_dir = image_path / "train"
test_dir = image_path / "valid"
train_data = torchvision.datasets.ImageFolder(train_dir, transform=effnetb2_transforms)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=effnetb2_transforms)
class_names = train_data.classes
train_dataloader_effnetb2 = DataLoader(train_data,
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       num_workers=NUM_WORKERS,
                                       pin_memory=True,
                                       )
test_dataloader_effnetb2 = DataLoader(test_data,
                                      batch_size=BATCH_SIZE,
                                      shuffle=False,
                                      num_workers=NUM_WORKERS,
                                      pin_memory=True,
                                      )

optimizer = torch.optim.Adam(params=effnetb2.parameters(), lr=1e-3)
# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss()

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [6]:
%%writefile engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch

  Returns:
    A tuple containing train_loss (float), train_accuracy (float).
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X_, y_) in enumerate(dataloader):
    # Send data to target device
    X, y = X_.to(device), y_.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate  and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X_, y_) in enumerate(dataloader):
      # Send data to target device
      X, y = X_.to(device), y_.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

      # Print out what's happening
    print(f"Epoch: {epoch+1} | "
    f"train_loss: {train_loss:.4f} | "
    f"train_acc: {train_acc:.4f} | "
    f"test_loss: {test_loss:.4f} | "
    f"test_acc: {test_acc:.4f}")

      # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing engine.py


In [7]:
epochs = 20
import engine
effnetb2_results = engine.train(model=effnetb2,
                                train_dataloader=train_dataloader_effnetb2,
                                test_dataloader=test_dataloader_effnetb2,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                device=device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.8967 | train_acc: 0.4647 | test_loss: 1.4817 | test_acc: 0.7387
Epoch: 2 | train_loss: 1.2791 | train_acc: 0.7303 | test_loss: 1.1319 | test_acc: 0.7984
Epoch: 3 | train_loss: 1.0115 | train_acc: 0.7706 | test_loss: 0.9496 | test_acc: 0.8199
Epoch: 4 | train_loss: 0.8567 | train_acc: 0.8007 | test_loss: 0.8590 | test_acc: 0.8219
Epoch: 5 | train_loss: 0.7650 | train_acc: 0.8099 | test_loss: 0.7870 | test_acc: 0.8168
Epoch: 6 | train_loss: 0.6905 | train_acc: 0.8296 | test_loss: 0.7280 | test_acc: 0.8277
Epoch: 7 | train_loss: 0.6637 | train_acc: 0.8304 | test_loss: 0.7003 | test_acc: 0.8375
Epoch: 8 | train_loss: 0.6016 | train_acc: 0.8548 | test_loss: 0.6679 | test_acc: 0.8402
Epoch: 9 | train_loss: 0.5704 | train_acc: 0.8651 | test_loss: 0.6525 | test_acc: 0.8434
Epoch: 10 | train_loss: 0.5356 | train_acc: 0.8592 | test_loss: 0.6372 | test_acc: 0.8414
Epoch: 11 | train_loss: 0.5068 | train_acc: 0.8720 | test_loss: 0.6271 | test_acc: 0.8453
Epoch: 12 | train_l

In [8]:
# Save the model
target_dir_path = Path("models")
target_dir_path.mkdir(parents=True, exist_ok=True)
model_name = "pretrained_effnetb2_food101tiny.pth"
model_save_path = target_dir_path / model_name
print(f"[INFO] Saving model to: {model_save_path}")
torch.save(obj=effnetb2.state_dict(), f=model_save_path)

[INFO] Saving model to: models/pretrained_effnetb2_food101tiny.pth


In [9]:
def create_vit_model(num_classes:int=3,
                     seed:int=42):
  """Creates a ViT-B/16 feature extractor model and transforms.
  Args:
    num_classes (int, optional): number of target classes. Defaults to 3.
    seed (int, optional): random seed value for output layer. Defaults to 42.

  Returns:
    model (torch.nn.Module): ViT-B/16 feature extractor model.
    transforms (torchvision.transforms): ViT-B/16 image transforms.
  """
  # Create ViT_B_16 pretrained weights, transforms and model
  weights = torchvision.models.ViT_B_16_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.vit_b_16(weights=weights)

  # Freeze all layers in model
  for param in model.parameters():
    param.requires_grad = False

  # Change classifier head to suit our needs (this will be trainable)
  torch.manual_seed(seed)
  model.heads = nn.Sequential(nn.Linear(in_features=768, # keep this the same as original model
                                        out_features=num_classes)) # update to reflect target number of classes

  return model, transforms

In [10]:
from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm

# 1. Create a function to return a list of dictionaries with sample, truth label, prediction, prediction probability and prediction time
def pred_and_store(paths: List[Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  # 2. Create an empty list to store prediction dictionaries
  pred_list = []

  # 3. Loop through target paths
  for path in tqdm(paths):

    # 4. Create empty dictionary to store prediction information for each sample
    pred_dict = {}

    # 5. Get the sample path and ground truth class name
    pred_dict["image_path"] = path
    class_name = path.parent.stem
    pred_dict["class_name"] = class_name

    # 6. Start the prediction timer
    start_time = timer()

    # 7. Open image path
    img = Image.open(path)

    # 8. Transform the image, add batch dimension and put image on target device
    transformed_image = transform(img).unsqueeze(0).to(device)

    # 9. Prepare model for inference by sending it to target device and turning on eval() mode
    model.to(device)
    model.eval()

    # 10. Get prediction probability, predicition label and prediction class
    with torch.inference_mode():
      pred_logit = model(transformed_image) # perform inference on target sample
      pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
      pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
      pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

      # 11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
      pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
      pred_dict["pred_class"] = pred_class

      # 12. End the timer and calculate time per pred
      end_time = timer()
      pred_dict["time_for_pred"] = round(end_time-start_time, 4)

    # 13. Does the pred match the true label?
    pred_dict["correct"] = class_name == pred_class

    # 14. Add the dictionary to the list of preds
    pred_list.append(pred_dict)

  # 15. Return list of prediction dictionaries
  return pred_list

# Make predictions across test dataset with EffNetB2
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_names=class_names,
                                          device="cpu") # make predictions on CPU

  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.correct.value_counts()

effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"Average time per prediction: {effnetb2_average_time_per_pred} s")

Average time per prediction: 0.1094 s


In [12]:
try:
  import gradio as gr
except:
  print("[INFO] Couldn't find gradio... installing it.")
  !pip install -q gradio
  import gradio as gr

print(f"gradio version: {gr.__version__}")

[INFO] Couldn't find gradio... installing it.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
gradio version: 5.12.0


In [13]:
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = effnetb2_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

In [14]:
# Run demo with Gradio
import random

title = "FoodVision (food-101-tiny)"
class_names_str = ", ".join(class_names)
description = f"An EfficientNetB2 feature extractor computer vision model to classify images of food. Classes: {class_names_str}."
article = "ref: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/). Model trained with [food-101-tiny](https://www.kaggle.com/datasets/msarmi9/food101tiny)."
example_list = [str(filepath) for filepath in random.sample(test_data_paths, k=3)]

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
demo.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09dc1dca75d3a4bd6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
# Below is for deployment to Hugging face
# URL: https://huggingface.co/spaces/rapianyo/foodvision_food101tiny
import shutil
from pathlib import Path

# Create FoodVision mini demo path
foodvision_food101tiny_demo_path = Path("demos/foodvision_food101tiny/")
if foodvision_food101tiny_demo_path.exists():
  shutil.rmtree(foodvision_food101tiny_demo_path)
  foodvision_food101tiny_demo_path.mkdir(parents=True, exist_ok=True)

foodvision_food101tiny_examples_path = foodvision_food101tiny_demo_path / "examples"
foodvision_food101tiny_examples_path.mkdir(parents=True, exist_ok=True)

# Copy example_list to the examples directory
for example in example_list:
  example_path = Path(example)
  destination = foodvision_food101tiny_examples_path / example_path.name
  print(f"[INFO] Copying {example_path} to {destination}")
  shutil.copy2(src=example_path, dst=destination)

# Create a destination path for our target model
effnetb2_foodvision_food101tiny_model_destination = foodvision_food101tiny_demo_path / model_name

# Try to move the file
try:
  print(f"[INFO] Attempting to move {model_save_path} to {effnetb2_foodvision_food101tiny_model_destination}")

  # Move the model
  shutil.move(src=model_save_path,
              dst=effnetb2_foodvision_food101tiny_model_destination)

  print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
  print(f"[INFO] No model found at {model_save_path}, perhaps its already been moved?")
  print(f"[INFO] Model exists at {effnetb2_foodvision_food101tiny_model_destination}: {effnetb2_foodvision_food101tiny_model_destination.exists()}")

[INFO] Copying data/food-101-tiny/valid/ramen/1698372.jpg to demos/foodvision_food101tiny/examples/1698372.jpg
[INFO] Copying data/food-101-tiny/valid/sushi/1032351.jpg to demos/foodvision_food101tiny/examples/1032351.jpg
[INFO] Copying data/food-101-tiny/valid/sushi/1203702.jpg to demos/foodvision_food101tiny/examples/1203702.jpg
[INFO] Attempting to move models/pretrained_effnetb2_food101tiny.pth to demos/foodvision_food101tiny/pretrained_effnetb2_food101tiny.pth
[INFO] Model move complete.


In [27]:
# write create_effnetb2_model (exactly the same as the one above)
%%writefile demos/foodvision_food101tiny/model.py
import torch
import torchvision

from torch import nn

def create_effnetb2_model(num_classes: int=10,
                          seed: int=1161,
                          dropout: float=0.3):
  """Creates an EfficientNetB2 feature extractor model and transforms.

  Args:
      num_classes: number of classes in the classifier head.
          Defaults: 10 (size of food-101-tiny).
      seed: random seed value.
      dropout: dropout rate of the classifier head.

  Returns:
      model: EffNetB2 feature extractor model.
      transforms: EffNetB2 image transforms.
  """

  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights)

  # Freezes all layers in base model
  for param in model.parameters():
    param.requires_grad = False

  # Changes classifier head
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
      # 1408: number of in_feature of effnetb2.classifier
  )

  device = "cuda" if torch.cuda.is_available() else "cpu"
  return model.to(device), transforms

Overwriting model.py


In [17]:
%%writefile demos/foodvision_food101tiny/app.py
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

SEED = 1161
class_names = ['apple_pie',
               'bibimbap',
               'cannoli',
               'edamame',
               'falafel',
               'french_toast',
               'ice_cream',
               'ramen',
               'sushi',
               'tiramisu'] # food101tiny_train_data.classes
class_names_str = ", ".join(class_names)
model_name = "pretrained_effnetb2_food101tiny.pth"
device = "cpu" # free on HuggingFace

# Create EffNetB2 model
effnetb2, effnetb2_transforms = create_effnetb2_model(seed=SEED)
effnetb2.load_state_dict(
    torch.load(
        f=model_name,
        map_location=torch.device(device),
    )
)

# Create predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = effnetb2_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

### Create Gradio app ###
title = "FoodVision (food-101-tiny)"
description = f"An EfficientNetB2 feature extractor computer vision model to classify images of food. Classes: {class_names_str}."
article = "ref: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/). Model trained with [food-101-tiny](https://www.kaggle.com/datasets/msarmi9/food101tiny)."
example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
demo.launch(debug=False, share=True)

Writing demos/foodvision_food101tiny/app.py


In [34]:
%%writefile demos/foodvision_food101tiny/requirements.txt
torch
torchvision
gradio

Overwriting demos/foodvision_food101tiny/requirements.txt


In [19]:
# Zip foodvision_big folder but exclude certain files
!cd demos/foodvision_food101tiny && zip -r ../foodvision_food101tiny.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

# Download the zipped FoodVision Big app (if running in Google Colab)
try:
    from google.colab import files
    files.download("demos/foodvision_food101tiny.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download()")

  adding: app.py (deflated 57%)
  adding: examples/ (stored 0%)
  adding: examples/1698372.jpg (deflated 0%)
  adding: examples/1203702.jpg (deflated 0%)
  adding: examples/1032351.jpg (deflated 1%)
  adding: model.py (deflated 55%)
  adding: pretrained_effnetb2_food101tiny.pth (deflated 8%)
  adding: requirements.txt (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>